In [20]:
from astropy.coordinates import SkyCoord 
from astropy.coordinates import Angle
from astroquery.vizier import Vizier
import astropy.coordinates as coord
from extinctions import reddening
from astropy.io import fits
import astropy.units as u
from io import BytesIO
import pandas as pd
import numpy as np
import extinctions
import extinction
import mechanize
import requests
import json

## Query Catalogues

In [21]:
def retrieve_SDSS(input_ra, input_dec, input_name):
    '''Queries the SDSS catalogue DR16 for galaxy matches on Ra and Dec, within a search radius of 
       +/- 2 arcsecs, returning a dictionary of u,g,r,i,z magnitudes and errors, or of zero values if 
       the target isn't a catalogue match.
       
       input: target coordinates in degrees and name.
       output: a dictionary of results and a data collection note.'''
    
    # Execute SQL query in SDSS.
    br = mechanize.Browser()
    br.open("http://skyserver.sdss.org/dr16/en/tools/search/sql.aspx")
    
    sql = f'''SELECT TOP 5    \
        ra, dec, u, err_u, g, err_g, r, err_r, i, err_i, z, err_z \
        FROM  PhotoPrimary p              \
        WHERE ra BETWEEN {input_ra - 0.0014} and {input_ra + 0.0014} 
        AND dec BETWEEN {input_dec - 0.0014} and {input_dec + 0.0014}
        '''
    
    br.select_form(name="sql")
    br['cmd'] = sql
    br['format']=['csv']
    response = br.submit()
    file_like = BytesIO(response.get_data())
    dataframe = pd.read_csv(file_like, skiprows=1)

    print('\nQuerying SDSS for host galaxy data...\n')

    if dataframe.empty:
        result_sdss = {f"SDSS_{band}": 0 for band in ['u', 'err_u', 'g', 'err_g', 'r', 'err_r', 'i', 'err_i', 'z', 'err_z']}
        sdss_note = {"SDSS": f'{input_name} has no SDSS data'}
    
    # closest match on sky
    else:
        c = coord.SkyCoord(ra=str(input_ra), dec=str(input_dec), unit=(u.deg, u.deg))
        cat_sdss = SkyCoord(ra=dataframe['ra']*u.degree, dec=dataframe['dec']*u.degree)
        idx, d2d, d3d = c.match_to_catalog_sky(cat_sdss)

        dataframe.drop(columns=['ra', 'dec'], axis=1, inplace=True)
        dataframe.rename(lambda x: f'SDSS_{x}', axis='columns', inplace=True)
        result_sdss = (dataframe.to_dict('records'))[idx]
        sdss_note = {"SDSS": f'{input_name} has SDSS data'}
    
    return result_sdss, sdss_note

In [22]:
def retrieve_pans(input_ra, input_dec):
    '''Queries the PanSTARRS catalogue DR2 (or DR1) for galaxy matches on Ra and Dec, within a search radius of 
       +/- 2 arcsecs, returning a dictionary of g,r,i,z,y magnitudes and errors, or of zero values if 
       the target isn't a catalogue match.
       
       input: target coordinates in degrees.
       output: a dictionary of results and a data collection note.'''
    
    print('\nQuerying PS1 for host galaxy data...\n')
    
    def get_data(data_release):
        '''aims to pull PanSTARRS data from DR2 but checks DR1 if needed'''
    
        queryurl = f'https://catalogs.mast.stsci.edu/api/v0.1/panstarrs/{data_release}/stack.json?'
        queryurl += 'ra='+str(input_ra)   
        queryurl += '&dec='+str(input_dec) 
        queryurl += '&radius=0.0014'        # Radius in degrees, 1 arcsec = 0.0003 deg (when looking for nearby objects want to look within a 4" radius because this is the aperture size?)
        queryurl += '&columns=[raStack,decStack,gKronMag,gKronMagErr,rKronMag,rKronMagErr,iKronMag,iKronMagErr,zKronMag,zKronMagErr,yKronMag,yKronMagErr,gPSFMag,gPSFMagErr]'
    
    
        query = requests.get(queryurl)
        results = query.json()
        return np.array(results['data'])

    def closest_match(data, data_release):
        '''finds the closest match to target in the returned PanSTARRS data'''
        
        # consider zero matches
        if data.size == 0:
            pans_note = {"PANS": ' has no PanSTARRS data'}
            return {}, pans_note
        
        # consider multiple matches and find closest match
        c = coord.SkyCoord(ra=str(input_ra), dec=str(input_dec), unit=(u.deg, u.deg))
        cat_pan = coord.SkyCoord(ra=data[:,0]*u.degree, dec=data[:,1]*u.degree)
        idx, d2d, d3d = c.match_to_catalog_sky(cat_pan)
        
        df = pd.DataFrame(data, columns =[
            'Ra','Dec','PSTARRS_g','PSTARRS_e_g','PSTARRS_r','PSTARRS_e_r',
            'PSTARRS_i','PSTARRS_e_i','PSTARRS_z','PSTARRS_e_z','PSTARRS_y','PSTARRS_e_y'
            ,'PS_gPSF','PS_e_gPSF'
        ])
        
        closest_match = (df.to_dict('records'))[idx] 
        closest_match_df = df.loc[(df['Ra'] == df['Ra'][idx]) & (df['Dec'] == df['Dec'][idx])] 
        closest_match_df.drop(columns=['Ra', 'Dec'], axis=1, inplace=True)
        
        pans_note = {"PANS": f' has PanSTARRS {data_release} data'}
        
        if len(closest_match_df) > 1:
            # select result with no -999 values
            zero_val = []
            
            for i in range(len(closest_match_df)):
                try: 
                    zero_val.append(closest_match_df.loc[i].value_counts()[-999])
                except:
                    zero_val.append(0)
                
            idx_z = np.argmin(zero_val)
            
            #closest_match_df.drop(columns=['Ra', 'Dec'], axis=1, inplace=True)
            result_pans = (closest_match_df.to_dict('records'))[idx_z]
            pans_note = {"PANS": f' has PanSTARRS {data_release} data'}
        
        else:
            result_pans = (closest_match_df).to_dict('records')[0]
            
        # Replace any negative values 
        for key in result_pans.keys():
            if result_pans[key] < 0: 
                result_pans[key] = 0
                pans_note = {"PANS": f' had -999.0 PanSTARRS {data_release} data'}
            
        return result_pans, pans_note
                    
    # Attempt to query DR2 
    result_pans, pans_note = closest_match(get_data('dr2'), 'dr2')
    
    # If no data from DR2, query DR1
    if not result_pans:
        result_pans, pans_note = closest_match(get_data('dr1'), 'dr1')
        
    # If no data from either
    if not result_pans:
        result_pans = {f"PSTARRS_{band}": 0 for band in ['g', 'e_g', 'r', 'e_r', 'i', 'e_i', 'z', 'e_z', 'y', 'e_y', 'PS_gPSF', 'PS_e_gPSF']}
        #pans_note = {"PANS": ' has no PanSTARRS data'}
          
    return result_pans, pans_note

In [23]:
def retrieve_2mass(input_ra, input_dec):
    '''Queries the 2MASS ESC catalogue, in ViZieR, for galaxy matches on Ra and Dec, within a search radius 
       of +/- 2 arcsecs, returning a dictionary of J,H,K magnitudes and errors, or of zero values if the target 
       isn't a catalogue match. 
       
       input: target ra, dec and name.
       output: a dictionary of results and a data collection note.'''

    # Create a coordinate object from input RA and Dec
    c = coord.SkyCoord(ra=str(input_ra), dec=str(input_dec), unit=(u.deg, u.deg))

    print('\nQuerying ViZieR for host galaxy data via 2MASS...\n')
    
    def get_data(catalog, columns):
        """Query a specified 2MASS catalog."""
        mass_catalog = Vizier(columns=columns, catalog=catalog)
        
        catalogue_result = mass_catalog.query_region(c, radius=Angle(0.0014, "deg"))
            
        return catalogue_result

    def process_data(result, ext=True):
        """Process and rename columns in the result."""
        
        if len(result) == 0:
            return None
        
        data = result[0]
        data.remove_columns(("RAJ2000", "DEJ2000"))
        prefix = "2MASS_"
        data.rename_columns(
            ["J.ext", "e_J.ext", "H.ext", "e_H.ext", "K.ext", "e_K.ext"] if ext else ["J.K20e", "e_J.K20e", "H.K20e", "e_H.K20e", "K.K20e", "e_K.K20e"],
            [f"{prefix}J", f"{prefix}e_J", f"{prefix}H", f"{prefix}e_H", f"{prefix}K", f"{prefix}e_K"]
        )
        
        df_data = data.to_pandas()
        
        result_mass = df_data.to_dict('records')[0]
        
        return result_mass

    def check_blending():
        """Check for blending with nearby objects using Pan-STARRS."""
        query = f'https://catalogs.mast.stsci.edu/api/v0.1/panstarrs/dr2/stack.json?ra={input_ra}&dec={input_dec}&radius=0.0012&columns=[raStack,decStack]'

        pan_query = requests.get(query)
        pan_results = pan_query.json()
        pan_data = np.array(pan_results['data'])
        
        return len(pan_data) > 1, pan_data
        
    # Query for extrapolated data
    ext_result = get_data("VII/233/xsc", ["RAJ2000", "DEJ2000", "J.ext", "e_J.ext", "H.ext", "e_H.ext", "K.ext", "e_K.ext"])
    
    result_mass = process_data(ext_result, ext=True)
    mass_note = {"2MASS": ' has 2MASS ESC extrapolated data'}

    # if there is no extrapolated data, or if it contains null values - query for isophotal data
    if result_mass is None or any(np.isnan(val) for val in result_mass.values()):
        iso_result = get_data("VII/233/xsc", ["RAJ2000", "DEJ2000", "J.K20e", "e_J.K20e", "H.K20e", "e_H.K20e", "K.K20e", "e_K.K20e"])
        result_mass = process_data(iso_result, ext=False)
        mass_note = {"2MASS": ' has 2MASS ESC isophotal data due to having no extrapolated data/ null values'}
        
    if result_mass is None:
        # Query for PSC data
        psc_result = get_data("II/246/out", ["Jstdap", "e_Jstdap", "Hstdap", "e_Hstdap", "Kstdap", "e_Kstdap"])

        if len(psc_result) == 1:
            has_blending, pan_data = check_blending()
            if has_blending:
                result_mass = {"2MASS_J": 0, "2MASS_e_J": 0, "2MASS_H": 0, "2MASS_e_H": 0, "2MASS_K": 0, "2MASS_e_K": 0}
                mass_note = {"2MASS": ' has no 2MASS ESC data and there is blending in the PSC'}
            else:
                result_mass = {"2MASS_J": 0, "2MASS_e_J": 0, "2MASS_H": 0, "2MASS_e_H": 0, "2MASS_K": 0, "2MASS_e_K": 0}
                mass_note = {"2MASS": ' has 2MASS PSC data but not using it in this study'}
        else:
            result_mass = {"2MASS_J": 0, "2MASS_e_J": 0, "2MASS_H": 0, "2MASS_e_H": 0, "2MASS_K": 0, "2MASS_e_K": 0}
            mass_note = {"2MASS": ' has no 2MASS data'}

    return result_mass, mass_note

In [24]:
def retrieve_legacy(input_ra, input_dec):
    '''Queries the Legacy catalogue for galaxy matches on Ra and Dec, within a search radius of 
       +/- 10 arcsecs, returning a dictionary of g,r,i,z and W1,W2,W3,W4 magnitudes and errors, or 
       of zero values if the target isn't a catalogue match. 
       
       input: target ra, dec and name.
       output: a dictionary of results and a data collection note.'''
    
    print('\nQuerying Legacy Survey for host galaxy data...\n')
    
    def flux_to_mag_leg(input_flux): 
        #Just a convience function to make a flux into a magnitude
        return 22.5 - 2.5*np.log10(input_flux)

    # Define matching range
    search_radius = 0.0014
    ralow = input_ra - search_radius
    rahigh = input_ra + search_radius
    declow = input_dec - search_radius
    dechigh = input_dec + search_radius

    # attempt to query legacy survey and define required fields 
    try: 
        url = f'https://www.legacysurvey.org/viewer/ls-dr10/cat.fits?ralo={ralow:.5f}&rahi={rahigh:.5f}&declo={declow:.5f}&dechi={dechigh:.5f}'
        hdu = fits.open(url)
        hdu_data = hdu[1].data
        
        fields = ['ra','dec','mjd_max','mjd_min','flux_g','flux_r', 'flux_i', 'flux_z', 
                  'flux_w1', 'flux_w2', 'flux_w3', 'flux_w4', 
                  'flux_ivar_g', 'flux_ivar_r', 'flux_ivar_i', 'flux_ivar_z', 
                  'flux_ivar_w1', 'flux_ivar_w2', 'flux_ivar_w3', 'flux_ivar_w4']
    
        #Match the catalogue to give the closest match to the input ra/dec
        #Remove any DUP sources which are GAIA duplicates with no fluxes
        c = SkyCoord(ra=input_ra*u.degree, dec=input_dec*u.degree)
        inds = (hdu_data['type'] != 'DUP')  
        catalog = SkyCoord(ra=hdu_data['ra'][inds]*u.degree, dec=hdu_data['dec'][inds]*u.degree) 
        idx, d2d, d3d = c.match_to_catalog_sky(catalog)
    
        #Create dictionary
        r = {}
        for field in fields:
            try:
                r[field] = []
                r[field].append(hdu_data[field][idx]) 
                if (field[0:4] == 'flux'):
                    if (field[5:9] != 'ivar'): 
                        r[field.replace('flux', 'mag')] = []
                        r[field.replace('flux', 'mag')].append(flux_to_mag_leg(hdu_data[field][idx]))
                if (field[0:9] == 'flux_ivar'): 
                    r[field.replace('flux_ivar', 'mag_err')] = []
                    flux_error = 1./np.sqrt(hdu_data[field][idx])
                    error_calc = 2.5/np.log(10)*(flux_error)/hdu_data[field.replace('flux_ivar', 'flux')][idx]
                    r[field.replace('flux_ivar', 'mag_err')].append(error_calc)
                
            except:
                r[field].append('-99.98') 
                if (field[0:4] == 'flux'):
                    if (field[5:9] != 'ivar'): 
                        r[field.replace('flux', 'mag')] = []
                        r[field.replace('flux', 'mag')].append(0)
                if (field[0:9] == 'flux_ivar'): 
                    r[field.replace('flux_ivar', 'mag_err')] = []
                    r[field.replace('flux_ivar', 'mag_err')].append(0)
        
        
        report_fields = ['mag_g','mag_r', 'mag_i', 'mag_z', 'mag_w1', 'mag_w2', 'mag_w3', 'mag_w4']
        result_legacy = {}
        legacy_note = {"LEGACY": ' has Legacy data'}
        
        for rfield in report_fields:
            if (rfield[0:3] == 'mag'):          
                if (np.isfinite(r[rfield][0])): 
                    if (r[rfield][0] > 0):      
                        result_legacy[rfield]=round(r[rfield][0],3) 
                        result_legacy[rfield.replace('mag_', 'mag_err_')] = round(r[rfield.replace('mag_', 'mag_err_')][0],4) # now instead of working with mag, you replace that and work with mag_err, rounding to 4dp
                        
                        # if error is larger than 0.5
                        if round(r[rfield.replace('mag_', 'mag_err_')][0],4) > 0.5: 
                                result_legacy[rfield] = 0 
                                result_legacy[rfield.replace('mag_', 'mag_err_')] = 0 
                                legacy_note = {"LEGACY": ' has Legacy data but some WISE has been removed due to large errors'}
                
                    # if magnitude value is zero        
                    else: 
                        result_legacy[rfield] = 0 
                        result_legacy[rfield.replace('mag_', 'mag_err_')] = 0 
                        legacy_note = {"LEGACY": ' has some Legacy data'}
    
    except: 
        result_legacy = {'mag_g': 0, 'mag_err_g': 0, 'mag_r': 0, 'mag_err_r': 0, 'mag_i': 0, 'mag_err_i': 0, 
                             'mag_z': 0, 'mag_err_z': 0, 'mag_w1': 0, 'mag_err_w1': 0, 'mag_w2': 0, 'mag_err_w2': 0, 
                             'mag_w3': 0, 'mag_err_w3': 0, 'mag_w4': 0, 'mag_err_w4': 0}
        legacy_note = {"LEGACY": ' has no Legacy data'}
     
    if len(result_legacy.keys()) < 16:
        zero_val = {'mag_g': 0, 'mag_err_g': 0, 'mag_r': 0, 'mag_err_r': 0, 'mag_i': 0, 'mag_err_i': 0, 
                             'mag_z': 0, 'mag_err_z': 0, 'mag_w1': 0, 'mag_err_w1': 0, 'mag_w2': 0, 'mag_err_w2': 0, 
                             'mag_w3': 0, 'mag_err_w3': 0, 'mag_w4': 0, 'mag_err_w4': 0}
        legacy_note = {"LEGACY": ' has some Legacy data'} 
        comb_dict ={k:v for d in (zero_val,result_legacy) for k,v in d.items()}
        comb_dict = {**zero_val,**result_legacy}
        result_legacy = comb_dict
            
    return result_legacy, legacy_note, len(result_legacy.keys())

In [25]:
def retrieve_galex(input_ra, input_dec):
    '''Queries the GALEX catalogue for galaxy matches on Ra and Dec, within a search radius of 
       +/- 2 arcsecs, returning a dictionary of FUV,NUV magnitudes and errors, or of zero values 
       if the target isn't a catalogue match.
       
       input: target ra and dec.
       output: a dictionary of results and a data collection note.'''

    #Make query to GALEX.
    galex = Vizier(columns=["RAJ2000", "DEJ2000","FUVmag", "e_FUVmag", "NUVmag", "e_NUVmag"], catalog="II/335/galex_ais")

    print('\nQuerying ViZieR for host galaxy data via GALEX...\n')
    
    c = coord.SkyCoord(ra=str(input_ra), dec=str(input_dec), unit=(u.deg, u.deg))
    gal_result = galex.query_region(c, radius=Angle(0.0014, "deg"))

    if len(gal_result) == 1:
        #Reduce the catalogue to the closest match to the input coordinates and return the dictionary results.
        gal_result = gal_result[0]
        cat_gal = SkyCoord(ra=gal_result['RAJ2000'], dec=gal_result['DEJ2000']) 
    
        idx, d2d, d3d = c.match_to_catalog_sky(cat_gal)
    
        gal_result.remove_columns(("RAJ2000", "DEJ2000"))
        gal_result.rename_columns(("FUVmag", "e_FUVmag", "NUVmag", "e_NUVmag"), ("GALEX_FUV", "GALEX_e_FUV", "GALEX_NUV", "GALEX_e_NUV"))
    
        df_gal = (gal_result.to_pandas())
        
        if df_gal.isnull().values.any():                      # Replace any NaN results with zeroes.
            remove_nan = df_gal.fillna(0)
            result_galex = remove_nan.to_dict('records')[idx] # Dictionary of closest match
            galex_note = {"GALEX": ' has some GALEX data'}
        
        else:
            result_galex = (df_gal.to_dict('records'))[idx]   # Dictionary of closest match
            galex_note = {"GALEX": ' has GALEX data'}
    
    if len(gal_result) == 0:
        result_galex = {'GALEX_FUV': 0, 'GALEX_e_FUV':0, 'GALEX_NUV': 0, 'GALEX_e_NUV': 0}
        galex_note = {"GALEX": ' has no GALEX data'}
    
    return result_galex, galex_note

## Extinction Correction

In [26]:
def ext(input_ra,input_dec):
    '''Calculates the extinction values for various wavelengths based on the given RA and Dec.
    
    input: target ra and dec 
    output: a dictionary of extinction corrected results'''
    
    red = (reddening.Reddening(input_ra, input_dec)).query_local_map(dustmap='sfd')*0.86
    AV = 3.1*float(str(red)[1:-1])
    
    # effective wavelengths for all filters
    wave = np.array([3587., 4717., 6165., 7476., 8923.,                # SDSS u, g, r, i, z
                     4810., 6170., 7520., 8660., 9620., 4810.,         # PS1 g, r, i, z, y, gPSF
                     12300., 16400., 21600.,                           # 2MASS J, H, K
                     4717., 6165., 7476., 8923.,                       # LEGACY g, r, i, z
                     34000., 46000., 120000., 220000.,                 # LEGACY W1,, W2, W3, W4
                     1516., 2267.])                                    # GALEX FUV, NUV
    

    # extinction keys for respective filters
    AC_keys = ['SDSS_u', 'SDSS_g', 'SDSS_r', 'SDSS_i', 'SDSS_z', 
               'PSTARRS_g', 'PSTARRS_r', 'PSTARRS_i', 'PSTARRS_z', 'PSTARRS_y', 'PS_gPSF', 
               '2MASS_J', '2MASS_H', '2MASS_K', 
               'mag_g', 'mag_r', 'mag_i', 'mag_z', 
               'mag_w1', 'mag_w2', 'mag_w3', 'mag_w4', 
               'GALEX_FUV', 'GALEX_NUV']
    
    AC = extinction.fitzpatrick99(wave, AV, 3.1)
    
    ext_val = {}
    for A, B in zip(AC_keys, AC):
        ext_val[A] = B
    
    return ext_val

## PSF/Kron Mag (B/T)g Estimate

In [27]:
def bulge_to_total(ra,dec):
    
    pans_data = retrieve_pans(ra,dec)
    
    if pans_data[0]['PSTARRS_g'] and pans_data[0]['PS_gPSF'] > 0:
        b_t = 10**(0.4*(pans_data[0]['PSTARRS_g'] - pans_data[0]['PS_gPSF']))
        e_1 = np.sqrt(pans_data[0]['PSTARRS_e_g']**2 + pans_data[0]['PS_e_gPSF']**2) 
        e_2 = 0.4*e_1
        b_t_e = 2.303*b_t*e_2
        
        
        bt_note = {"(B/T)g": f' (B/T)g = {np.round(b_t,3)} (+/- {np.round(b_t_e,3)}) derived using PanSTARRS PSF/Kron Mags'}
        
    else:
        bt_note = {"(B/T)g": ' (B/T)g cannot be calculated with PS1 data'}
        
    return(bt_note)

## Collect Data for Targets 

In [ ]:
# read in .txt file of targets (target_name,ra,dec,z)
with open('path/to/txt-file') as file:
    name = []
    z = []
    ra = []
    dec = []
    
    for line in file:
        data = line.split(',')
        name.append(data[0])
        ra.append(float(data[8]))
        dec.append(float(data[9]))
        z.append(float(data[10]))


for i in range(len(name)):
    
    # dictionary of raw data
    raw_data = retrieve_SDSS(ra[i], dec[i], name[i])[0] | retrieve_pans(ra[i], dec[i])[0] | retrieve_2mass(ra[i], dec[i])[0] | retrieve_legacy(ra[i], dec[i])[0] | retrieve_galex(ra[i],dec[i])[0]
    print(f'Raw Data Collected for {name[i]}.')

    # dictionary of data collection notes
    data_notes = retrieve_SDSS(ra[i], dec[i], name[i])[1] | retrieve_pans(ra[i], dec[i])[1] | retrieve_2mass(ra[i], dec[i])[1] | retrieve_legacy(ra[i], dec[i])[1] | retrieve_galex(ra[i],dec[i])[1] | bulge_to_total(ra[i],dec[i])
    print(f'Collection Notes Compiled for {name[i]}.')    
        
    # adding a systematic error
    syst_err = dict.fromkeys(['SDSS_err_u','SDSS_err_g','SDSS_err_r','SDSS_err_i','SDSS_err_z',
                              'PSTARRS_e_g','PSTARRS_e_r','PSTARRS_e_i','PSTARRS_e_z','PSTARRS_e_y',
                              '2MASS_e_J','2MASS_e_H','2MASS_e_K',
                              'mag_err_g', 'mag_err_r', 'mag_err_i', 'mag_err_z',
                              'mag_err_w1','mag_err_w2','mag_err_w3','mag_err_w4',
                              'GALEX_e_FUV', 'GALEX_e_NUV'], 0.1)
    
    syst_data = {key: np.sqrt(raw_data[key]**2 + syst_err.get(key, 0)**2)         
                        for key in raw_data.keys()}
    
    # zero values remain zero
    for key, value in raw_data.items():
        if value <= 0:
            syst_data[key] = 0                                          
     
    
    # dictionary of extinction corrected results
    ext_values = ext(ra[i], dec[i])                                     
    extcor_data = {key: syst_data[key] - ext_values.get(key, 0)         
                           for key in syst_data.keys()}
    
    # zero values remain zero
    for key, value in extcor_data.items():
        if value < 0:
            extcor_data[key] = 0                                        
    
    
    # AB magnitude conversion
    if extcor_data['2MASS_J'] > 0:
        extcor_data['2MASS_J'] = extcor_data['2MASS_J'] + 0.91
    
    if extcor_data['2MASS_H'] > 0:
        extcor_data['2MASS_H'] = extcor_data['2MASS_H'] + 1.39
    
    if extcor_data['2MASS_K'] > 0:
        extcor_data['2MASS_K'] = extcor_data['2MASS_K'] + 1.85
    
    
    # rounding to 4dp
    arr = np.array(list(extcor_data.values()))
    rounded_arr = np.round(arr, 4)

    
    # dictionary of final data values
    fin_data = {key : rounded_arr[idx] for idx, key in enumerate(extcor_data)}
    
    # build .json
    tde = {f'{name[i]}': 
            {"redshift": f'{z[i]}', 
             "type":"TDE", 
             "photometry": [], "notes": [] }}

    keys = list(fin_data.keys())

    for key in keys:
        if (key[0:4] == 'SDSS'):
            if (key[5:6] != 'e'):
                if fin_data[key] and fin_data['SDSS_err_' + f'{key[5]}'] > 0: # if both magnitude and error on magnitude are larger than 0.
                    tde[f'{name[i]}']["photometry"].append({"band":f"{key[5]}", "mag": f"{fin_data[f'{key}']}", "err": f"{fin_data['SDSS_err_' + f'{key[5]}']}", "limit": False, "instrument": "", "telescope":"SDSS"})
    
        if (key[0:7] == 'PSTARRS'):
            if (key[8:9] != 'e'):
                if fin_data[key] > 0:
                    tde[f'{name[i]}']["photometry"].append({"band":f"{key[8]}", "mag": f"{fin_data[f'{key}']}", "err": f"{fin_data['PSTARRS_e_' + f'{key[8]}']}", "limit": False, "instrument": "", "telescope":"PS1"})
                
        if (key[0:5] == '2MASS'):
            if (key[6:7] != 'e'):
                if fin_data[key] > 0:
                    tde[f'{name[i]}']["photometry"].append({"band":f"{key[6]}", "mag": f"{fin_data[f'{key}']}", "err": f"{fin_data['2MASS_e_' + f'{key[6]}']}", "limit": False, "instrument": "", "telescope":"2MASS"})

        if (key[0:3] == 'mag'):
            if (key[4:7] != 'err'):
                if fin_data[key] > 0:
                    if (key[4] != 'w'):
                        tde[f'{name[i]}']["photometry"].append({"band":f"{key[4]}", "mag": f"{fin_data[f'{key}']}", "err": f"{fin_data['mag_err_' + f'{key[4]}']}", "limit": False, "instrument": "", "telescope":"DECam"})
                    else:
                        if fin_data['mag_err_w' +f'{key[5]}'] <= 0.5:
                            tde[f'{name[i]}']["photometry"].append({"band":f"W{key[5]}", "mag": f"{fin_data[f'{key}']}", "err": f"{fin_data['mag_err_w' + f'{key[5]}']}", "limit": False, "instrument": "", "telescope":"WISE"})
        
        if (key[0:5] == 'GALEX'):
            if (key[6:7] != 'e'):
                if fin_data[key] > 0:
                    tde[f'{name[i]}']["photometry"].append({"band":f"{key[6:9]}", "mag": f"{fin_data[f'{key}']}", "err": f"{fin_data['GALEX_e_' + f'{key[6:9]}']}", "limit": False, "instrument": "", "telescope":"GALEX"})

 
    tde[f'{name[i]}']["notes"].append(f"{data_notes['SDSS']},{data_notes['PANS']},{data_notes['2MASS']},{data_notes['LEGACY']},{data_notes['GALEX']},{data_notes['(B/T)g']}")

    with open(f'path/to/data/folder', 'w') as outfile:
        json.dump(tde, outfile, indent=2)


Querying SDSS for host galaxy data...


Querying PS1 for host galaxy data...



/var/folders/x7/yl76x1s54ml4dckx50h4dfm9mztcwv/T/ipykernel_68053/4101882693.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  closest_match_df.drop(columns=['Ra', 'Dec'], axis=1, inplace=True)



Querying ViZieR for host galaxy data via 2MASS...


Querying Legacy Survey for host galaxy data...


Querying ViZieR for host galaxy data via GALEX...

Raw Data Collected for NGC1332.

Querying SDSS for host galaxy data...


Querying PS1 for host galaxy data...



/var/folders/x7/yl76x1s54ml4dckx50h4dfm9mztcwv/T/ipykernel_68053/4101882693.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  closest_match_df.drop(columns=['Ra', 'Dec'], axis=1, inplace=True)



Querying ViZieR for host galaxy data via 2MASS...


Querying Legacy Survey for host galaxy data...


Querying ViZieR for host galaxy data via GALEX...


Querying PS1 for host galaxy data...



...

/var/folders/x7/yl76x1s54ml4dckx50h4dfm9mztcwv/T/ipykernel_68053/4101882693.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  closest_match_df.drop(columns=['Ra', 'Dec'], axis=1, inplace=True)



Querying ViZieR for host galaxy data via 2MASS...


Querying Legacy Survey for host galaxy data...


Querying ViZieR for host galaxy data via GALEX...

Raw Data Collected for NGC3379.

Querying SDSS for host galaxy data...


Querying PS1 for host galaxy data...



/var/folders/x7/yl76x1s54ml4dckx50h4dfm9mztcwv/T/ipykernel_68053/4101882693.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  closest_match_df.drop(columns=['Ra', 'Dec'], axis=1, inplace=True)



Querying ViZieR for host galaxy data via 2MASS...


Querying Legacy Survey for host galaxy data...


Querying ViZieR for host galaxy data via GALEX...


Querying PS1 for host galaxy data...

Collection Notes Compiled for NGC3379.
INFO: Loading the maps from /Users/pxr754/.extinction/maps//maps.yaml
 -  sfd is loaded


/var/folders/x7/yl76x1s54ml4dckx50h4dfm9mztcwv/T/ipykernel_68053/4101882693.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  closest_match_df.drop(columns=['Ra', 'Dec'], axis=1, inplace=True)
/Users/pxr754/opt/anaconda3/lib/python3.9/site-packages/extinctions/reddening.py:53: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.maps = yaml.load(open(pmap))
